In [1]:
import os
import time
import sys
import warnings

sys.path.append("/home/prakank/anaconda3/lib/python3.8/site-packages/")

import scipy
import numpy as np
from cvxopt import matrix, solvers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from python.svmutil import *

In [2]:
BINARY_CLASSIFICATION = True
LAST_DIGIT = 1
BASE_DIR = "../"
train_path = os.path.join(BASE_DIR, "data", "mnist","train.csv")
test_path  = os.path.join(BASE_DIR, "data", "mnist","test.csv")

In [3]:
def load_data(filename, Binary):
    data = np.genfromtxt(filename,delimiter=',')
    data_x = data[:,:784]/255
    data_y = data[:,784]
    data_y = data_y.reshape((data_y.shape[0],1))
    
    if Binary:
        data_x = data_x[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = data_y[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = -1.0*(data_y==LAST_DIGIT) + 1.0*(data_y==((LAST_DIGIT+1)%10))
        
    return data_x,data_y

In [24]:
def linear_kernel(X,y):
    mat = np.array(X*y)
    return np.dot(mat, mat.T)

def gaussian_kernel_element(X1,X2,gamma):
    return np.exp(-(np.linalg.norm(X1-X2)**2) * gamma)

def gaussian_kernel(X,gamma):
    X_sq   = np.sum(np.multiply(X, X),axis=1, keepdims=True)
    kernel_partial = X_sq + X_sq.T
    kernel_partial = kernel_partial - 2*np.dot(X,X.T)
    kernel = np.power(np.exp(-gamma),kernel_partial)
    return kernel

    kernel = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            kernel[i,j] = gaussian_kernel_element(X[i],X[j],gamma)
    return Kernel

In [46]:
class SVM:
    def __init__(self,kernel,C,threshold=1e-5,gamma=0.05):
        if kernel == "linear":
            self.kernel = linear_kernel
        else:
            self.kernel = gaussian_kernel
            
        self.C = float(C)
        self.threshold = threshold
        self.gamma = gamma
    
    def train(self, X_train, Y_train):
        # minimizing function
        P = 0
        if self.kernel == linear_kernel:
            P = self.kernel(X_train,Y_train)
        elif self.kernel == gaussian_kernel:
            kernel = matrix(self.kernel(X_train, self.gamma))
            P = (kernel*Y_train)*(Y_train.T)

        P = matrix(.5 * (P + P.T))  # Just to be on the safe side (ensuring P is symmetric)
        q = matrix(-1.0*np.ones((X_train.shape[0],1)))
        c = 0.0
        
        # Inequalities
        pos = np.diag(np.ones(X_train.shape[0]))
        neg = np.diag(-np.ones(X_train.shape[0]))
        G   = matrix( np.vstack((neg,pos)) )
        
        zer   = np.zeros((X_train.shape[0]))
        c_val = self.C*np.ones(X_train.shape[0])
        h     = matrix(np.concatenate((zer,c_val)))
        
        # Equality
        A = matrix(Y_train.reshape((1,Y_train.shape[0])))
        b = matrix(0.0)
        
        solvers.options['show_progress'] = True
        sol = solvers.qp(P, q, G, h, A, b);
        
        alpha = np.array(sol['x'])
        self.support_vector_flag = (alpha > self.threshold).reshape(-1)
        # self.support_vector_indices = (np.arange(len(alpha)))[self.support_vector_flag]
        self.alpha = alpha[self.support_vector_flag]
        self.support_vector_x = X_train[self.support_vector_flag]
        self.support_vector_y = Y_train[self.support_vector_flag]
        
        # return self.support_vector_x, self.support_vector_y, self.alpha
        
        if self.kernel == linear_kernel:
            w_partial = self.support_vector_x * self.support_vector_y
            self.w = np.sum(w_partial * self.alpha, axis=0)
            
            b1 = np.min(X_train[(Y_train == 1).reshape(-1)] * self.w)
            b2 = np.max(X_train[(Y_train ==-1).reshape(-1)] * self.w)
            self.b  = (b1+b2)*(-0.5)
        else:
            self.w = None
            
            b1 = float("inf")
            b2 = -float("inf")
            
            for i in range(len(self.alpha)):
                val = 0
                for j in range(len(self.alpha)):
                    val += self.alpha[j] * self.support_vector_y[j] * gaussian_kernel_element(self.support_vector_x[j], self.support_vector_x[i], gamma=self.gamma)
                if self.support_vector_y[i] == 1:
                    b1 = min(b1, val)
                else:
                    b2 = max(b2,val)
            if b1 == float("inf"):
                b1 = 0
            if b2 == -float("inf"):
                b2 = 0                
            self.b = (b1+b2)*(-0.5)
            
        return self.alpha, self.w, self.b

    def predict(self, X_test):   
        if self.kernel == linear_kernel:
            Y_pred = (np.dot(X_test,self.w)) + self.b
        else:
            Y_pred = np.zeros((X_test.shape[0]))
            alpha  = self.alpha.reshape(-1)            
            support_vector_y = self.support_vector_y.reshape(-1)
            
            for i in range(X_test.shape[0]):
                mat = np.array(list(map(lambda x: gaussian_kernel_element(x,X_test[i],gamma=self.gamma),self.support_vector_x)))
                Y_pred[i] = np.sum(mat*alpha*support_vector_y) + self.b
                # val = 0
                # for index in range(len(self.alpha)):
                #     val += self.alpha[index] \
                #             *self.support_vector_y[index] \
                #             *gaussian_kernel_element(self.support_vector_x[index],X_test[i],gamma=self.gamma)
                # val += self.b
                # Y_pred[i] = val
                
        Y_pred = Y_pred.reshape(-1)
        Y_pred = np.array(list(map(lambda x: -1 if x<0 else 1,Y_pred)))
        return Y_pred

In [6]:
if BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)        

In [326]:
svm_lin = SVM(kernel="linear",C=1)
alpha,w,b = svm_lin.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
print("W:",w,"\n")
print("b:",b)

In [330]:
Y_validation = svm_lin.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_lin.predict(X_test)
print("Test Set Accuracy (Linear Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 97.975%
Test Set Accuracy (Linear Kernel): 98.477%


In [47]:
svm_gau = SVM(kernel="gaussian",C=1)
alpha,w,b = svm_gau.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
print("W:",w,"\n")
print("b:",b)

     pcost       dcost       gap    pres   dres
 0: -7.9991e+01 -5.6970e+03  3e+04  2e+00  2e-15
 1: -4.0391e+01 -2.4332e+03  3e+03  1e-01  2e-15
 2: -4.7583e+01 -4.1104e+02  4e+02  1e-02  3e-15
 3: -6.9765e+01 -1.9362e+02  1e+02  3e-03  2e-15


In [45]:
Y_validation = svm_gau.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_gau.predict(X_test)
print("Test Set Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867, 1)
(867,)
()
(867,) (867, 1) (867

In [33]:
Y_validation = svm_gau.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_gau.predict(X_test)
print("Test Set Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 99.825%
Test Set Accuracy (Gaussian Kernel): 95.847%


In [11]:
# print(X_train)
# X2 = np.multiply(X_train, X_train)
X2 = np.sum(np.multiply(X_train, X_train),axis=1, keepdims=True)
print(X2)
# for i in range(X_train.shape[0]):
#     for j in range(X_train.shape[1]):
#         if X_train[i][j]!=0:
#             print(X_train[i][j], X2[i])
print(X2.shape)

[[ 79.77453287]
 [ 76.39240292]
 [ 95.26622068]
 ...
 [151.19277201]
 [ 54.54472895]
 [126.95784698]]
(4000, 1)


In [335]:
X3 = np.dot(X_train,X_train.T)

In [336]:
print(X3.shape)

(4000, 4000)


In [338]:
if (X2 == X3).all():
    print("EWEU")

In [339]:
print(X2)

[[159.54906574 156.16693579 175.04075356 ... 230.96730488 134.31926182
  206.73237985]
 [156.16693579 152.78480584 171.65862361 ... 227.58517493 130.93713187
  203.3502499 ]
 [175.04075356 171.65862361 190.53244137 ... 246.4589927  149.81094963
  222.22406767]
 ...
 [230.96730488 227.58517493 246.4589927  ... 302.38554402 205.73750096
  278.15061899]
 [134.31926182 130.93713187 149.81094963 ... 205.73750096 109.0894579
  181.50257593]
 [206.73237985 203.3502499  222.22406767 ... 278.15061899 181.50257593
  253.91569396]]


In [340]:
print(X3)

[[ 79.77453287  43.20298347  40.49613226 ...  43.47472511  44.13780854
   49.48364475]
 [ 43.20298347  76.39240292  39.68292195 ...  47.69900807  40.75952326
   34.24224529]
 [ 40.49613226  39.68292195  95.26622068 ...  76.97261053  34.34415994
   64.42574394]
 ...
 [ 43.47472511  47.69900807  76.97261053 ... 151.19277201  30.23266436
   88.29900807]
 [ 44.13780854  40.75952326  34.34415994 ...  30.23266436  54.54472895
   32.68659746]
 [ 49.48364475  34.24224529  64.42574394 ...  88.29900807  32.68659746
  126.95784698]]
